# Building compiled GRIDCERF technology layers

## 1. Setup environment

### 1.1 Download GRIDCERF


Download the GRIDCERF package if you have not yet done so from here: https://doi.org/10.57931/2281697. Please extract GRIDCERF inside the data directory of this repository as the paths in this notebook are set to that expectation.

### 1.2 Import necessary Python packages

In [6]:
import os
import glob

import yaml
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio


## 2. Configuration


In [12]:
# get the parent directory path to where this notebook is currently stored
root_dir = os.path.dirname(os.getcwd())

# data directory in repository
data_dir = os.path.join(root_dir, "data")

# GRIDCERF data directory from downloaded archive
gridcerf_dir = os.path.join(data_dir, "gridcerf")

# common exclusion layers
common_dir = os.path.join(gridcerf_dir, 'common')

# directory to write compiled layers to
compiled_dir = os.path.join(gridcerf_dir, 'compiled')

# directory containing technology specific layers
tech_dir = os.path.join(gridcerf_dir, 'technology_specific')

# directory containing reference data
reference_dir = os.path.join(gridcerf_dir, 'reference')

# a file of technologies and their levels with the raster name
cerf_technologies_file = os.path.join(reference_dir, "gridcerf_technologies.txt")

siting_mask_file_0 = os.path.join(reference_dir, "gridcerf_sitingmask_0-value.tif") 
siting_mask_file_1 = os.path.join(reference_dir, "gridcerf_sitingmask.tif")

# technology layer list
layer_list_file = os.path.join(reference_dir, 'technology_layer_list.csv')

output_dir =os.path.join(compiled_dir, 'compiled_technology_layers')

# create a list of ssp scenarios to create suitability data for
ssp_list = ['ssp2', 'ssp3', 'ssp5']

# create a list of technology groupings to create composite suitability layers for
tech_group_list =  ['geothermal', 'gas', 'nuclear',  'solar', 'solar_csp', 'wind', 'wind_offshore', 'biomass', 'coal', 'refinedliquids']

# create a list of years to create composite suitability layers for
year_list = list(range(2020, 2100+1, 5))

## 3. Generate compiled technology suitability rasters

### 3.1 Functions to build suitability


In [8]:
def compile_layer(target_layer, compile_list, siting_mask_file_0, siting_mask_file_1, output_dir):
    """Compile a suitability layer."""
    
    land_mask_0 = rasterio.open(siting_mask_file_0).read(1)

    with rasterio.open(siting_mask_file_1) as template:

        metadata = template.meta.copy()

        land_mask_1 = template.read(1)

        for index, i in enumerate(compile_list):

            with rasterio.open(i) as src:

                if index == 0:
                    arr = src.read(1)
                else:
                    arr += src.read(1)

            # adjust to binary suitability with nodata set to 3 for uint8 storage 
            arr = np.where(arr == 0, 0, 1)

            # apply land mask
            arr *= land_mask_1
            
            arr += land_mask_0

            # write compiled file
            output_file = os.path.join(output_dir, target_layer)
            with rasterio.open(output_file, 'w', **metadata) as dest:

                dest.write(arr.astype(np.int16), 1)
                
                                       
    return output_file


### 3.2 Process layers

## Create a combined common layer

In [9]:
# get a list of all common layers
common_files = glob.glob(os.path.join(common_dir, "*.tif"))

n_common = len(common_files)

print(f"There are {n_common} common layers.")
for file in os.listdir(common_dir):
    print(file)

There are 24 common layers.
gridcerf_dod_military_installations_conus.tif
gridcerf_usfs_national_wild_scenic_river_system_conus.tif
gridcerf_blm_nlcs_national_monument_conus.tif
gridcerf_blm_nlcs_wilderness_conus.tif
gridcerf_usfws_critical_habitat_conus.tif
gridcerf_wdpa_tnc_managed_lands_conus.tif
gridcerf_blm_historic_parks.tif
gridcerf_nps_historic_trails_conus.tif
gridcerf_usfs_wilderness_areas_conus.tif
gridcerf_usfs_national_designated_areas_conus.tif
gridcerf_usfws_special_designation_conus.tif
gridcerf_blm_areas_of_critical_environmental_concern.tif
gridcerf_usgs_padus_class_1_to_2_conus_additions.tif
gridcerf_nlcd_wetlands_conus.tif
gridcerf_blm_drecp_wildlife_allocation.tif
gridcerf_nps_administrative_boundaries_conus.tif
gridcerf_blm_national_register_property.tif
gridcerf_usgs_lakes_reservoirs.tif
gridcerf_combined_non_padus.tif
gridcerf_bor_surface_management_agency_areas.tif
gridcerf_epa_class1_airsheds_conus.tif
gridcerf_nps_scenic_trails_conus.tif
gridcerf_blm_historic

In [10]:
target_layer = 'combined_common.tif'
output_dir = '../data/gridcerf/compiled/combined_common'
compile_layer(target_layer=target_layer, compile_list=common_files, 
              siting_mask_file_0=siting_mask_file_0, siting_mask_file_1=siting_mask_file_1, output_dir=output_dir)

'../data/gridcerf/compiled/combined_common/combined_common.tif'

## Create Composite Technology Suitability Layers

In [11]:
def create_compile_dict(ssp, year, layer_df_path=layer_list_file,
                        layer_type_list=['common', 'reference', 'technology_specific', 'scenario_specific'],
                        ssp_year_dependency_list=['gridcerf_densely_populated_50k_per25sqmi',
                                                 'gridcerf_densely_populated_50k_per25sqmi_25buffer',
                                                 'gridcerf_densely_populated_nuclear']):

    """ Creates a compile dictionary based on input file, ssp_list, and year_list. Can specify
    which layers should have specific ssp and year suffixes added on."""

    compile_dict = {}

    # read in layer data file
    ssp_year_df = pd.read_csv(layer_df_path)

    for tech in ssp_year_df.columns[2:]:

        tech_df = ssp_year_df.copy()

        for layer_type in layer_type_list:

            # reduce to layer type
            layer_df = tech_df[tech_df['layer_type'] == layer_type].copy()

            if layer_type == 'scenario_specific':
                for layer in ssp_year_dependency_list:
                    type = layer.split("populated_")[1]
                    gridcerf_layer_name = f'gridcerf_population_{ssp}_{year}_1km_conus_{type}'
                    layer_df['layer'].replace(layer, gridcerf_layer_name, inplace=True)
            else:
                pass

            # collect tech_group
            tech_group = tech.split("_")[1]
            
            if tech.split("_")[2] in ['csp', 'offshore']:
                tech_group = tech.split("_")[1] + "_" + tech.split("_")[2]
            else:
                tech_group = tech.split("_")[1]
                
            # collect layers
            layer_df = layer_df[layer_df[tech] == 1]

            layer_compile_list = layer_df['layer'].to_list()


            if compile_dict.get(ssp) is None:
                compile_dict.update({ssp: {year: {tech_group: {tech: {layer_type: layer_compile_list}}}}})

            elif compile_dict[ssp].get(year) is None:
                compile_dict[ssp].update({year: {tech_group: {tech: {layer_type: layer_compile_list}}}})

            elif compile_dict[ssp][year].get(tech_group) is None:
                compile_dict[ssp][year].update({tech_group: {tech: {layer_type: layer_compile_list}}})

            elif compile_dict[ssp][year][tech_group].get(tech) is None:
                compile_dict[ssp][year][tech_group].update({tech: {layer_type: layer_compile_list}})

            else:
                compile_dict[ssp][year][tech_group][tech].update({layer_type: layer_compile_list})

    return compile_dict


def compile_layer(target_layer, compile_list, siting_mask_file_0, siting_mask_file_1, output_dir):
    """Compile a suitability layer."""

    land_mask_0 = rasterio.open(siting_mask_file_0).read(1)

    with rasterio.open(siting_mask_file_1) as template:

        metadata = template.meta.copy()

        land_mask_1 = template.read(1)

        for index, i in enumerate(compile_list):

            with rasterio.open(i) as src:

                if index == 0:
                    arr = src.read(1).astype(np.int64)
                else:
                    arr += src.read(1).astype(np.int64)

            # adjust to binary suitability with nodata set to 3 for uint8 storage
            arr = np.where(arr == 0, 0, 1)

            # apply land mask
            arr *= land_mask_1

            arr += land_mask_0

            # write compiled file
            output_file = os.path.join(output_dir, target_layer)
            with rasterio.open(output_file, 'w', **metadata) as dest:

                dest.write(arr.astype(np.int16), 1)


    return output_file

def compile_techs(year, ssp_list, tech_group_list):

    # create folders
    for ssp in ssp_list:

        for technology_group in tech_group_list:
            print(ssp, year, technology_group)

            # create compile dictionary
            compile_dict = create_compile_dict(ssp=ssp, year=year)

            # create output folders
            ssp_folder_path = os.path.join(output_dir,ssp)
            if os.path.exists(ssp_folder_path):
                pass
            else:
                os.mkdir(ssp_folder_path)

            year_folder_path = os.path.join(ssp_folder_path, str(year))
            if os.path.exists(year_folder_path):
                pass
            else:
                os.mkdir(year_folder_path)


            techgroup_folder_path = os.path.join(year_folder_path, technology_group)
            if os.path.exists(techgroup_folder_path):
                pass
            else:
                os.mkdir(techgroup_folder_path)

            for technology in compile_dict[ssp][year][technology_group]:
                print(f'Compiling {technology}')
                compile_list = []
                for layer_type in compile_dict[ssp][year][technology_group][technology]:
                    for layer in compile_dict[ssp][year][technology_group][technology][layer_type]:

                        layer_path = os.path.join(gridcerf_dir, layer_type, f'{layer}.tif')
                        compile_list.append(layer_path)

                compile_layer(target_layer=technology,
                                       compile_list=compile_list,
                                       siting_mask_file_0=siting_mask_file_0,
                                       siting_mask_file_1=siting_mask_file_1,
                                       output_dir=techgroup_folder_path)

    return technology


In [ ]:
%%time
for year in year_list:
    compile_techs(year=year, ssp_list=ssp_list, tech_group_list=tech_group_list)

ssp2 2020 geothermal
Compiling gridcerf_geothermal_centralized_enhanced_dry-hybrid_class1.tif
Compiling gridcerf_geothermal_centralized_enhanced_recirculating_class1.tif
Compiling gridcerf_geothermal_centralized_enhanced_dry-hybrid_class2.tif
Compiling gridcerf_geothermal_centralized_enhanced_recirculating_class2.tif
Compiling gridcerf_geothermal_centralized_enhanced_dry-hybrid_class3.tif
Compiling gridcerf_geothermal_centralized_enhanced_recirculating_class3.tif
Compiling gridcerf_geothermal_centralized_enhanced_dry-hybrid_class4.tif
Compiling gridcerf_geothermal_centralized_enhanced_recirculating_class4.tif
Compiling gridcerf_geothermal_centralized_enhanced_dry-hybrid_class5.tif
Compiling gridcerf_geothermal_centralized_enhanced_recirculating_class5.tif
ssp2 2020 gas
Compiling gridcerf_gas_cc_ccs_dry.tif
Compiling gridcerf_gas_cc_ccs_oncethrough.tif
Compiling gridcerf_gas_cc_ccs_recirculating.tif
Compiling gridcerf_gas_cc_ccs_recirculating-seawater.tif
Compiling gridcerf_gas_cc_no-cc